# 카카오 로컬 API 활용
### 위도, 경도 찾기

In [8]:
import requests
from urllib.parse import quote
import pandas as pd

- API Key 가져오기

In [2]:
from google.colab import files
up = files.upload()

Saving kakaoapikey.txt to kakaoapikey.txt


In [3]:
with open('kakaoapikey.txt') as kf:
    kakao_key = kf.read()

- url 만들기

In [5]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json' # api 주소.. 
addr = '광주광역시 서구 내방로 111(치평동)'
url = f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C%20%EC%84%9C%EA%B5%AC%20%EB%82%B4%EB%B0%A9%EB%A1%9C%20111%28%EC%B9%98%ED%8F%89%EB%8F%99%29'

In [6]:
# API Key를 입력하기 위한 header 만들기 
# 카카오 개발 가이드 sample에서 확인 가능
header = {'Authorization':f'KakaoAK {kakao_key}'}

- 요청해서 결과 얻기

In [7]:
result = requests.get(url, headers = header).json()
result

{'documents': [{'address': {'address_name': '광주 서구 치평동 1200',
    'b_code': '2914012000',
    'h_code': '2914074500',
    'main_address_no': '1200',
    'mountain_yn': 'N',
    'region_1depth_name': '광주',
    'region_2depth_name': '서구',
    'region_3depth_h_name': '치평동',
    'region_3depth_name': '치평동',
    'sub_address_no': '',
    'x': '126.851461925213',
    'y': '35.1600994105234'},
   'address_name': '광주 서구 내방로 111',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '광주 서구 내방로 111',
    'building_name': '광주광역시청',
    'main_building_no': '111',
    'region_1depth_name': '광주',
    'region_2depth_name': '서구',
    'region_3depth_name': '치평동',
    'road_name': '내방로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.851461925213',
    'y': '35.1600994105234',
    'zone_no': '61945'},
   'x': '126.851461925213',
   'y': '35.1600994105234'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [9]:
result.keys()

dict_keys(['documents', 'meta'])

In [11]:
len(result['documents'])

1

In [13]:
result['documents'][0].keys()
# x, y가 경도, 위도

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [19]:
latitude = result['documents'][0]['y']
longitude = result['documents'][0]['x']
latitude, longitude

('35.1600994105234', '126.851461925213')

- 광주광역시 주요 행정기관 좌표
<br>
- 도로명주소 api로 찾아낸 도로명주소를 이용하면 될 것 같다.


In [22]:
places = '광주광역시청,광주광역시 동구청,광주광역시 북구청,광주광역시 서구청,광주광역시 남구청,광주광역시 광산구청'.split(',')
addrs = ['광주광역시 서구 내방로 111(치평동)',
 '광주광역시 동구 서남로 1(서석동)',
 '광주광역시 북구 우치로 77(용봉동)',
 '광주광역시 서구 경열로 33(농성동)',
 '광주광역시 남구 봉선로 1(봉선동)',
 '광주광역시 광산구 광산로29번길 15(송정동)']
 # 앞서 api를 이용해 구한 도로명 주소

In [26]:
XYs = []
for idx, addr in enumerate(addrs):
    local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    url = f'{local_url}?query={quote(addr)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers = header).json()
    latitude = float(result['documents'][0]['y'])
    longitude = float(result['documents'][0]['x'])
    XYs.append([places[idx], addr, latitude, longitude])
XYs

[['광주광역시청', '광주광역시 서구 내방로 111(치평동)', 35.1600994105234, 126.851461925213],
 ['광주광역시 동구청', '광주광역시 동구 서남로 1(서석동)', 35.1461417002004, 126.92309316107],
 ['광주광역시 북구청', '광주광역시 북구 우치로 77(용봉동)', 35.1742905296566, 126.912107567985],
 ['광주광역시 서구청', '광주광역시 서구 경열로 33(농성동)', 35.1520290256721, 126.890112188537],
 ['광주광역시 남구청', '광주광역시 남구 봉선로 1(봉선동)', 35.1329064682079, 126.902447908848],
 ['광주광역시 광산구청',
  '광주광역시 광산구 광산로29번길 15(송정동)',
  35.1396474365046,
  126.79364427408]]

In [27]:
df = pd.DataFrame(XYs, columns = ['행정기관', '도로명주소', '위도', '경도'])
df

,행정기관,도로명주소,위도,경도
0,광주광역시청,광주광역시 서구 내방로 111(치평동),35.160099,126.851462
1,광주광역시 동구청,광주광역시 동구 서남로 1(서석동),35.146142,126.923093
2,광주광역시 북구청,광주광역시 북구 우치로 77(용봉동),35.174291,126.912108
3,광주광역시 서구청,광주광역시 서구 경열로 33(농성동),35.152029,126.890112
4,광주광역시 남구청,광주광역시 남구 봉선로 1(봉선동),35.132906,126.902448
5,광주광역시 광산구청,광주광역시 광산구 광산로29번길 15(송정동),35.139647,126.793644


In [28]:
df.to_csv('광주광역시_행정기관_좌표.csv', index=False, encoding='euc-kr')

선생님 ver

In [29]:
# 앞서 저장했던 도로명주소 csv 파일을 불러와서 사용
up = files.upload()

Saving 광주광역시_주요_행정기관2.csv to 광주광역시_주요_행정기관2.csv


In [33]:
df = pd.read_csv('광주광역시_주요_행정기관.csv')
df

,장소,도로명주소
0,광주광역시청,광주광역시 서구 내방로 111(치평동)
1,광주광역시 동구청,광주광역시 동구 서남로 1(서석동)
2,광주광역시 북구청,광주광역시 북구 우치로 77(용봉동)
3,광주광역시 서구청,광주광역시 서구 경열로 33(농성동)
4,광주광역시 남구청,광주광역시 남구 봉선로 1(봉선동)
5,광주광역시 광산구청,광주광역시 광산구 광산로29번길 15(송정동)


In [39]:
# 추가
df2 = pd.read_csv('광주광역시_주요_행정기관2.csv', encoding='euc-kr') # 저장할때 썼던 인코딩 써야함.
df2

,장소,도로명주소
0,광주광역시청,광주광역시 서구 내방로 111(치평동)
1,광주광역시 동구청,광주광역시 동구 서남로 1(서석동)
2,광주광역시 북구청,광주광역시 북구 우치로 77(용봉동)
3,광주광역시 서구청,광주광역시 서구 경열로 33(농성동)
4,광주광역시 남구청,광주광역시 남구 봉선로 1(봉선동)
5,광주광역시 광산구청,광주광역시 광산구 광산로29번길 15(송정동)


In [36]:
lng_list, lat_list = [],[]
for i in df.index:
    addr=df['도로명주소'][i]
    url = f'{local_url}?query={quote(addr)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers = header).json()
    latitude = float(result['documents'][0]['y'])
    longitude = float(result['documents'][0]['x'])
    lng_list.append(longitude)
    lat_list.append(latitude)
lng_list, lat_list

([126.851461925213,
  126.92309316107,
  126.912107567985,
  126.890112188537,
  126.902447908848,
  126.79364427408],
 [35.1600994105234,
  35.1461417002004,
  35.1742905296566,
  35.1520290256721,
  35.1329064682079,
  35.1396474365046])

In [40]:
df['위도'] = lat_list
df['경도'] = lng_list
df
# 기존에 있던 df에 추가하면 편함

,장소,도로명주소,위도,경도
0,광주광역시청,광주광역시 서구 내방로 111(치평동),35.160099,126.851462
1,광주광역시 동구청,광주광역시 동구 서남로 1(서석동),35.146142,126.923093
2,광주광역시 북구청,광주광역시 북구 우치로 77(용봉동),35.174291,126.912108
3,광주광역시 서구청,광주광역시 서구 경열로 33(농성동),35.152029,126.890112
4,광주광역시 남구청,광주광역시 남구 봉선로 1(봉선동),35.132906,126.902448
5,광주광역시 광산구청,광주광역시 광산구 광산로29번길 15(송정동),35.139647,126.793644


In [43]:
df.to_csv('광주광역시_주요기관_위도경도.csv', index=False)